In [1]:
import time
start_time = time.time()

In [2]:
import nltk
import scipy
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import numpy as np
import re
import pickle

Split Incoming Enteries

In [3]:
ETITLE = ["ENDTITLE"]
def split_text(text):
    entries = []
    j=0
    while (j <(len(text))):
        if (text[j].count('ENDTITLE') == 3):
            text_split = re.split("ENDTITLE|ENDTITLE|ENDTITLES|ENDABSTRACTS",text[j])
            title1 = text_split[0].strip()
            title2 = text_split[1].strip()
            title3 = text_split[2].strip()
            abstract = text_split[3].replace("\r\n"," ").strip()
            decision = text_split[4].strip()
            entries.append({u'title':title1,u'abstract':abstract,u'decision':decision})
            j = j +1
        elif (set(ETITLE).intersection(text[j].split()) == set(['ENDTITLE'])):
            text_split = re.split("ENDTITLES|ENDTITLE|ENDABSTRACTS",text[j])
            title1 = text_split[0].strip()
            title2 = text_split[1].strip()
            abstract = text_split[2].replace("\r\n"," ").strip()
            decision = text_split[3].strip()
            entries.append({u'title':title1,u'abstract':abstract,u'decision':decision})
            j = j +1
        else: 
            text_split = re.split("ENDTITLES|ENDABSTRACTS",text[j])
            title = text_split[0].strip()
            abstract = text_split[1].replace("\r\n"," ").strip()
            decision = text_split[2].strip()
            entries.append({u'title':title,u'abstract':abstract,u'decision':decision})
            j = j + 1
    return(entries)

Create n-grams

In [4]:
def BIGRAMS(string):
    token = nltk.word_tokenize(string)
    bigram = ngrams(token,2)
    bigrams = ', '.join(' '.join((a, b)) for a, b in bigram)
    biNoSpace = bigrams.replace(" ","")
    BIGRAM = biNoSpace.replace(",", " ")
    return BIGRAM
    
def TRIGRAMS(string):
    token = nltk.word_tokenize(string)
    trigram = ngrams(token, 3)
    trigrams = ', '.join(' '.join((a, b, c)) for a, b, c in trigram)
    triNoSpace = trigrams.replace(" ","")
    TRIGRAM = triNoSpace.replace(",", " ")
    return TRIGRAM

def QUADGRAMS(string):
    token = nltk.word_tokenize(string)
    quadgram = ngrams(token, 4)
    quadgrams = ', '.join(' '.join((a, b, c, d)) for a, b, c, d in quadgram)
    quadNoSpace = quadgrams.replace(" ","")
    QUADGRAM = quadNoSpace.replace(",", " ")  
    return QUADGRAM

Load any csv's

In [5]:
def load(name):
    with open(name+".csv", "rb") as f:
        File = np.loadtxt(f, dtype=str)
    return File

Function to Search:

In [6]:
def Search(Abs, W):
    ListOfWords = []
    match = set(Abs).intersection(set(W))
    Match = list(match)
    i=0
    while (i < len(Match)):
        ListOfWords.append(Match[i])
        i = i + 1
    return ListOfWords

thing to tie it all together:

In [7]:
def Key_Num(TEXT, DICT):
    S1 = Search(TEXT.split()             ,DICT)
    S2 = Search(BIGRAMS(TEXT).split()    ,DICT)
    S3 = Search(TRIGRAMS(TEXT).split()   ,DICT)
    S4 = Search(QUADGRAMS(TEXT).split()  ,DICT)
    
    score1 = len(S1)
    score2 = len(S2)
    score3 = len(S3)
    score4 = len(S4)
    total = sum([score1, score2, score3 , score4])
    
    return [score1, score2, score3, score4, total]

function to import data, and function to remove non ascii characters:

In [8]:
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

Data = []
with open("small.txt", "r") as ins:
    for line in ins:
        line = _removeNonAscii(line)
        line = line.replace(",", "")
        line = line.replace(".", "")
        Data.append(line)      

Lets run it!!!!!!!! (gets messier after this, maybe clean it up later?)

In [9]:
TEXT = split_text(Data)

TITLE = []
ABSTRACT = []
DECISION = []

for r in range(len(Data)):
    TITLE.append(TEXT[r].get('title').lower())
    ABSTRACT.append(TEXT[r].get('abstract').lower())
    DECISION.append(TEXT[r].get('decision').lower())

key = load("KeyWords")
q=0
while (q < len(key)):
    key[q] = key[q].lower()
    q = q + 1

In [10]:
RESULTS = [] 
for i in range(len(Data)):
    RESULTS.append(Key_Num(TITLE[i], key))
    RESULTS.append(Key_Num(ABSTRACT[i], key))
    RESULTS.append(DECISION[i])

for n, i in enumerate(RESULTS):
    if i == "rejected" :
        RESULTS[n] = 0
    if i == "core" :
        RESULTS[n] = 2
    if i == "non-core" :
        RESULTS[n] = 1

RESULT_array=np.reshape(np.hstack(RESULTS), (len(Data),11))

not every entry tell us if it is core, so it sucks and we can get rid of it:

In [11]:
RESULTS_ARRAY = []
m = 0
while (m < len(RESULT_array)):
    if (RESULT_array[m][10] != ''):
        RESULTS_ARRAY.append(RESULT_array[m])
    m = m + 1

function to save everything:

In [12]:
def output_file(name,output):
    with open(name+".pkl",'wb') as f:
        pickle.dump(output,f)
        
output_file('keyword_occurances',RESULTS_ARRAY)

In [13]:
with open('keyword_occurances.pkl','rb') as f:
    mydata= pickle.load(f)

In [14]:
print("My program took", time.time() - start_time, "to run")

('My program took', 109.19884300231934, 'to run')
